In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import os
from google.colab import drive
from tqdm import tqdm
import collections

# === 1. 설정 (기존과 동일) ===
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

ROOT_PATH = '/content/drive/MyDrive/Capstone_Project'
DATA_PATH = os.path.join(ROOT_PATH, 'processed_data', 'train_data_30_10.npy')
LABEL_PATH = os.path.join(ROOT_PATH, 'processed_data', 'train_label_30_10.npy')
MODEL_SAVE_BASE = os.path.join(ROOT_PATH, 'experiment_full_metric')

CLASSES = ['Walking', 'Shopping', 'Falldown', 'Threat']
BATCH_SIZE = 32
EPOCHS = 30
LEARNING_RATE = 0.01
NUM_RUNS = 5
step_size = 15


# === 2. 데이터 로드 ===
print("▶ 데이터 로딩...")
X = np.load(DATA_PATH)
Y = np.load(LABEL_PATH)

class_weights = compute_class_weight('balanced', classes=np.unique(Y), y=Y)
class_weights = torch.tensor(class_weights, dtype=torch.float32)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === 3. 모델 정의 (여기에 ST-GCN 또는 LSTM 클래스 코드 반드시 포함!) ===
# class Graph:
#     def __init__(self, num_node=17):
#         self.num_node = num_node
#         self.edges = [(0,1),(0,2),(1,3),(2,4),(5,7),(7,9),(6,8),(8,10),(5,6),(5,11),(6,12),(11,12),(11,13),(13,15),(12,14),(14,16)]
#         self.A = self.get_adjacency_matrix(self.edges, self.num_node)
#     def get_adjacency_matrix(self, edges, num_node):
#         A = np.zeros((num_node, num_node))
#         for i in range(num_node): A[i, i] = 1
#         for edge in edges:
#             A[edge[0], edge[1]] = 1
#             A[edge[1], edge[0]] = 1
#         row_sum = np.sum(A, axis=1)
#         D_inv_sqrt = np.power(row_sum, -0.5)
#         D_inv_sqrt[np.isinf(D_inv_sqrt)] = 0.
#         D_mat = np.diag(D_inv_sqrt)
#         return torch.tensor(D_mat.dot(A).dot(D_mat), dtype=torch.float32)

# class STGCNBlock(nn.Module):
#     def __init__(self, in_channels, out_channels, A, stride=1, residual=True):
#         super().__init__()
#         self.A = A
#         self.gcn = nn.Conv2d(in_channels, out_channels, kernel_size=1)
#         self.tcn = nn.Sequential(
#             nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True),
#             # Kernel size (9,1)은 T가 9보다 작으면 에러날 수 있으나 padding=4가 있어서 T=1도 처리가능
#             nn.Conv2d(out_channels, out_channels, kernel_size=(9,1), padding=(4,0), stride=(stride,1)),
#             nn.BatchNorm2d(out_channels), nn.Dropout(0.5, inplace=True)
#         )
#         if not residual: self.residual_conv = lambda x: 0
#         elif (in_channels == out_channels) and (stride == 1): self.residual_conv = lambda x: x
#         else: self.residual_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=(stride,1))
#     def forward(self, x):
#         x_spatial = self.gcn(x)
#         N, C, T, V = x_spatial.size()
#         x_spatial = torch.matmul(x_spatial.view(N, C*T, V), self.A.to(x.device)).view(N, C, T, V)
#         return torch.relu(self.tcn(x_spatial) + self.residual_conv(x))

# class ActionRecognitionModel(nn.Module):
#     def __init__(self, num_classes, in_channels=3, num_frames=30, num_joints=17):
#         super().__init__()
#         graph = Graph(num_node=num_joints)
#         self.A = nn.Parameter(graph.A, requires_grad=False)
#         self.data_bn = nn.BatchNorm1d(in_channels * num_joints)

#         # [수정됨 3] 경량화 채널 적용 (Half-STGCN)
#         # 64-128-256 구조를 -> 32-64-128 구조로 변경하여 파라미터와 연산량 감소
#         self.st_gcn_networks = nn.Sequential(
#             STGCNBlock(in_channels, 32, self.A),  # 64 -> 32
#             STGCNBlock(32, 32, self.A),           # 64 -> 32
#             STGCNBlock(32, 32, self.A),           # 64 -> 32
#             STGCNBlock(32, 64, self.A, stride=2), # 64 -> 32, 128 -> 64
#             STGCNBlock(64, 64, self.A),           # 128 -> 64
#             STGCNBlock(64, 64, self.A),           # 128 -> 64
#             STGCNBlock(64, 128, self.A, stride=2),# 128 -> 64, 256 -> 128
#             STGCNBlock(128, 128, self.A),         # 256 -> 128
#             STGCNBlock(128, 128, self.A)          # 256 -> 128
#         )
#         # 마지막 FC Layer 입력 채널도 128로 감소
#         self.fcn = nn.Conv2d(128, num_classes, kernel_size=1)

#     def forward(self, x):
#         N, C, T, V, M = x.size()
#         x = x.mean(dim=4) # (N, C, T, V)
#         x = x.permute(0, 1, 3, 2).contiguous().view(N, C*V, T)
#         x = self.data_bn(x)
#         x = x.view(N, C, V, T).permute(0, 1, 3, 2).contiguous()
#         x = self.st_gcn_networks(x)
#         # Global Average Pooling (T, V 차원에 대해 평균)
#         x = torch.nn.functional.avg_pool2d(x, x.size()[2:])
#         return self.fcn(x).view(x.size(0), -1)

# class Graph:
#     def __init__(self, num_node=17):
#         self.num_node = num_node
#         self.edges = [(0,1),(0,2),(1,3),(2,4),(5,7),(7,9),(6,8),(8,10),(5,6),(5,11),(6,12),(11,12),(11,13),(13,15),(12,14),(14,16)]
#         self.A = self.get_adjacency_matrix(self.edges, self.num_node)
#     def get_adjacency_matrix(self, edges, num_node):
#         A = np.zeros((num_node, num_node))
#         for i in range(num_node): A[i, i] = 1
#         for edge in edges:
#             A[edge[0], edge[1]] = 1
#             A[edge[1], edge[0]] = 1
#         row_sum = np.sum(A, axis=1)
#         D_inv_sqrt = np.power(row_sum, -0.5)
#         D_inv_sqrt[np.isinf(D_inv_sqrt)] = 0.
#         D_mat = np.diag(D_inv_sqrt)
#         return torch.tensor(D_mat.dot(A).dot(D_mat), dtype=torch.float32)

# class STGCNBlock(nn.Module):
#     def __init__(self, in_channels, out_channels, A, stride=1, residual=True):
#         super().__init__()
#         self.A = A
#         self.gcn = nn.Conv2d(in_channels, out_channels, kernel_size=1)
#         self.tcn = nn.Sequential(
#             nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True),
#             nn.Conv2d(out_channels, out_channels, kernel_size=(9,1), padding=(4,0), stride=(stride,1)),
#             nn.BatchNorm2d(out_channels), nn.Dropout(0.5, inplace=True)
#         )
#         if not residual: self.residual_conv = lambda x: 0
#         elif (in_channels == out_channels) and (stride == 1): self.residual_conv = lambda x: x
#         else: self.residual_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=(stride,1))
#     def forward(self, x):
#         x_spatial = self.gcn(x)
#         N, C, T, V = x_spatial.size()
#         x_spatial = torch.matmul(x_spatial.view(N, C*T, V), self.A.to(x.device)).view(N, C, T, V)
#         return torch.relu(self.tcn(x_spatial) + self.residual_conv(x))

# class ActionRecognitionModel(nn.Module):
#     def __init__(self, num_classes, in_channels=3, num_frames=30, num_joints=17):
#         super().__init__()
#         graph = Graph(num_node=num_joints)
#         self.A = nn.Parameter(graph.A, requires_grad=False)
#         self.data_bn = nn.BatchNorm1d(in_channels * num_joints)
#         self.st_gcn_networks = nn.Sequential(
#             STGCNBlock(in_channels, 64, self.A), STGCNBlock(64, 64, self.A), STGCNBlock(64, 64, self.A),
#             STGCNBlock(64, 128, self.A, stride=2), STGCNBlock(128, 128, self.A), STGCNBlock(128, 128, self.A),
#             STGCNBlock(128, 256, self.A, stride=2), STGCNBlock(256, 256, self.A), STGCNBlock(256, 256, self.A)
#         )
#         self.fcn = nn.Conv2d(256, num_classes, kernel_size=1)
#     def forward(self, x):
#         N, C, T, V, M = x.size()
#         x = x.mean(dim=4)
#         x = x.permute(0, 1, 3, 2).contiguous().view(N, C*V, T)
#         x = self.data_bn(x)
#         x = x.view(N, C, V, T).permute(0, 1, 3, 2).contiguous()
#         x = self.st_gcn_networks(x)
#         x = torch.nn.functional.avg_pool2d(x, x.size()[2:])
#         return self.fcn(x).view(x.size(0), -1)

#[Option 2] LSTM (비교 모델) 실험 시 사용 (이걸로 교체!)
# === 3. 모델 정의 (Baseline LSTM) ===
class BaselineLSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size=128, num_layers=2):
        super(BaselineLSTM, self).__init__()
        # batch_first=True: 입력이 (Batch, Time, Feature) 형태임
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # 입력 x 형태: (N, C, T, V, M) -> 5차원 텐서
        N, C, T, V, M = x.size()

        # 1. 차원 변경: (N, T, C, V, M) 순서로 바꿈 (시간 축을 앞으로)
        x = x.permute(0, 2, 1, 3, 4).contiguous()

        # 2. Flatten: (N, T, C*V*M) -> (N, T, 51)
        # 공간적 구조(Graph)를 무시하고 하나의 벡터로 펼침 -> 이것이 LSTM의 한계점 논리!
        x = x.view(N, T, -1)

        # 3. LSTM 통과
        # out: (Batch, Time, Hidden)
        # hidden: (Layers, Batch, Hidden)
        lstm_out, (hidden, cell) = self.lstm(x)

        # 4. 마지막 타임스텝의 결과만 사용하여 분류
        # Many-to-One 구조
        last_output = lstm_out[:, -1, :]
        return self.fc(last_output)


# === 4. 결과 저장소 초기화 ===
# 각 지표별로 리스트를 만들어 저장 (Key: Class -> Metric -> List of values)
metrics_storage = {
    cls: {'precision': [], 'recall': [], 'f1-score': []}
    for cls in CLASSES
}
metrics_storage['Overall'] = {'accuracy': []} # 전체 정확도 따로 저장

print(f"\n🚀 총 {NUM_RUNS}회 반복 실험 시작 (모든 지표 수집)")

for run in range(1, NUM_RUNS + 1):
    print("-" * 60)
    print(f"🔄 Run {run}/{NUM_RUNS}")

    # 데이터 분할
    X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=None, stratify=Y)
    train_data = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
    val_data = TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).long())
    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    # 모델 초기화 (★ 모델명 확인!)
    model = ActionRecognitionModel(num_classes=len(CLASSES)).to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size, gamma=0.1)

    best_acc = 0.0
    best_model_path = f"{MODEL_SAVE_BASE}_{run}.pth"

    # 학습
    for epoch in range(EPOCHS):
        # 1. Training
        model.train()
        running_loss = 0.0 # Loss 누적 변수 초기화

        # tqdm으로 진행바 표시 (leave=False로 설정하여 완료 후 줄바꿈 방지 깔끔하게)
        train_loop = tqdm(train_loader, desc=f"   Epoch {epoch+1}/{EPOCHS}", leave=False)

        for inputs, labels in train_loop:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            train_loop.set_postfix(loss=loss.item()) # 진행바 옆에 실시간 Loss 표시

        # Epoch 평균 Loss 계산
        avg_train_loss = running_loss / len(train_loader)

        # 2. Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        acc = 100 * correct / total
        scheduler.step()

        # ★ [추가된 부분] 중간 결과 출력 (Loss 및 Acc 확인)
        print(f"   [Run {run}] Epoch {epoch+1}/{EPOCHS} | Train Loss: {avg_train_loss:.4f} | Val Acc: {acc:.2f}%")

        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), best_model_path)
            # 최고 기록 갱신 시 옆에 표시해주면 좋음
            print(f"      └── Best Accuracy Updated! ({best_acc:.2f}%)")

    # 평가 (Best Model 로드)
    print(f"\n   ✅ Run {run} Finished. Final Best Acc: {best_acc:.2f}%")

    model.load_state_dict(torch.load(best_model_path))
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    # Classification Report 저장
    report = classification_report(all_labels, all_preds, target_names=CLASSES, output_dict=True)

    metrics_storage['Overall']['accuracy'].append(report['accuracy'])
    for cls in CLASSES:
            metrics_storage[cls]['precision'].append(report[cls]['precision'])
            metrics_storage[cls]['recall'].append(report[cls]['recall'])
            metrics_storage[cls]['f1-score'].append(report[cls]['f1-score'])

    print(f"   ✅ Run {run} Best Acc: {best_acc:.2f}% | Fall Recall: {report['Falldown']['recall']:.4f}")

# === 5. 최종 종합 리포트 출력 ===
print("\n" + "=" * 80)
print(f"📊 {NUM_RUNS}회 반복 실험 종합 결과 (Mean ± Std)")
print("=" * 80)

# 1. 전체 정확도
acc_mean = np.mean(metrics_storage['Overall']['accuracy']) * 100
acc_std = np.std(metrics_storage['Overall']['accuracy']) * 100
print(f"🏆 Overall Accuracy: {acc_mean:.2f} ± {acc_std:.2f}%")
print("-" * 80)

# 2. 클래스별 상세 테이블
header = f"{'Class':<12} | {'Metric':<10} | {'Mean':<8} | {'Std':<8}"
print(header)
print("-" * 45)

for cls in CLASSES:
    for metric in ['precision', 'recall', 'f1-score']:
        values = metrics_storage[cls][metric]
        mean_val = np.mean(values)
        std_val = np.std(values)

        # 보기 좋게 출력
        print(f"{cls:<12} | {metric.capitalize():<10} | {mean_val:.4f}   | ±{std_val:.4f}")
    print("-" * 45)

print("\n[논문 작성 가이드]")
print("1. Table 2에는 각 클래스의 F1-Score와 Recall을 중심으로 기재하세요.")
print(f"2. 특히 'Fall Recall'이 {np.mean(metrics_storage['Falldown']['recall']):.4f}로 높다는 점을 강조하세요.")

▶ 데이터 로딩...

🚀 총 5회 반복 실험 시작 (모든 지표 수집)
------------------------------------------------------------
🔄 Run 1/5


   [Run 1] Epoch 1/30 | Train Loss: 0.8590 | Val Acc: 59.54%
      └── Best Accuracy Updated! (59.54%)


   [Run 1] Epoch 2/30 | Train Loss: 0.6138 | Val Acc: 54.66%


   Epoch 3/30:  25%|██▌       | 400/1595 [00:08<00:23, 51.44it/s, loss=0.532]